In [ ]:
%%time
arcpy.env.parallelProcessingFactor = "60%"    #difinindo processamento paralelo para priorizar essa atividade
import arcpy

def copy_raster(in_raster, out_rasterdataset, pixel_type="16_BIT_UNSIGNED", format="TIFF"): #definir profundidade de cores  de acordo com profundidae podemos identificar mais ou menos cores ou valores do espectro
    
    """
    Função para copiar um raster usando arcpy.management.CopyRaster.

    Parameters:
    in_raster (str): Caminho do raster de entrada.
    out_rasterdataset (str): Caminho do raster de saída.
    pixel_type (str): Tipo de pixel (padrão é "32_BIT_FLOAT").
    format (str): Formato de saída (padrão é "COG"). # COG cloud optmized GEOTIFF
    """
    
    try:
        # Chama a ferramenta CopyRaster com os parâmetros fornecidos
        arcpy.management.CopyRaster(
            in_raster=in_raster,
            out_rasterdataset=out_rasterdataset,
            config_keyword="",
            background_value=None,
            nodata_value="",
            onebit_to_eightbit="NONE",
            colormap_to_RGB="NONE",
            pixel_type=pixel_type,
            scale_pixel_value="NONE",
            RGB_to_Colormap="NONE",
            format=format,
            transform="NONE",
            process_as_multidimensional="CURRENT_SLICE",
            build_multidimensional_transpose="NO_TRANSPOSE"
        )
        print(f"Raster copiado com sucesso de {in_raster} para {out_rasterdataset}.")
    except Exception as e:
        print(f"Erro ao copiar o raster: {e}")

# Exemplo de uso da função


In [ ]:
%%time
import arcpy # importar  biblioteca arcpy

In [ ]:
%%time
from arcpy import AIO  #	Usando este objeto, você pode executar tarefas como copiar arquivos de um local para outro, renomear ou excluir uma pasta, além de operações relacionadas a arquivos. 

In [ ]:
import arcpy

arcpy.env.workspace = "D:\cicatriz\planetary_earth"#  fazer uma busca pela tabela na coluna TILE Sentinel 2
fc = "pontos_s2_ERJ.shp"

try:
    with arcpy.da.SearchCursor(fc, "Name") as cursor:
        for row in cursor:
            print(row)
    print("Dados impressos com sucesso!")
except Exception as e:
    print(f"Erro: {e}")

In [ ]:
# busca individual por tile e orbita
%%time
tile = '23KPQ' # variaveis de filtragem por tile
orbit= 95   # variaveis de filtragem por orbit
cloud= 10   # variaveis de filtragem por cobertura de nuvem

In [ ]:
%%time
#Dicionario de busca
query = {
    "collections": ["sentinel-2-l2a"], # Sentinel-2 level 2A  (https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a)
   
    "datetime": "2025-07-01/2025-07-30", #  filtro por periodo de bsuca
    "query" :{"s2:mgrs_tile":{"eq":tile}}, #filtrar por TILE
    "cloudq": {"eo:cloud_cover": {"lt": 10}},# filtro por nuvem
    
    "limit": 20 # Maximo de resultados
          }

In [ ]:
%%time 

rc = arcpy.ia.RasterCollection.fromSTACAPI(stac_api="https://planetarycomputer.microsoft.com/api/stac/v1", # vai connecatar api do planetarycomputer de um rastercollection para manipular  e filtrar os dados
                                  query=query, # diconario anterior
                                  attribute_dict={   #dicionario que mostra  os metadados das imagens no retornaram da cosulat do stac stac
                                     
                                      "Name":"id",
                                      "Cloud Cover":"eo:cloud_cover",
                                      "StdTime":"datetime",             
                                      "Spatial Reference":"proj:epsg",
                                      "Orbita": "sat:relative_orbit",
                                      "cod":"s2:mgrs_tile"
                    
                                       
                                    })

In [ ]:
rcc=rc.filterByAttribute("Cloud Cover", "LESS_THAN", 30) #filtrado o dados as informações da cloud

In [ ]:
%%time
# organiza a tabela  por data e orbita
rcc=rcc.sort("StdTime").sort("Orbita")

rcc # imprimindo a tabela com as info


In [ ]:
%%time
import os
def compcic(item):
    raster=item['Raster']
    raster_obj=item['Raster']
    Nm= item["Name"][11:19]
    Nm= str(Nm)
    SRO=item["Name"][:4]
    SRO= str(SRO)
    print(SRO)
    print(Nm)
    nbr_output_path= os.path.join(r"D:\ProjetosARCPRO\MyProject.gdb", f"{SRO}{Nm}{tile}{orbit}")
    
    SWIR, NIR, RED, RED_E=raster_obj.getRasterBands(["B12","B8A","B4","B8"])
    
    rasters = [SWIR ,RED_E , RED, NIR]
    
    CB=arcpy.management.CompositeBands(
           in_rasters=rasters,
            out_raster=nbr_output_path
        )
    return {"raster":CB, "Name": item['Name'], "StdTime": item['StdTime'],"Cloud Cover": item['Cloud Cover'], "COD": item['cod'] ,"Orb":item ["Orbita"] }
      

In [ ]:
%%time
with arcpy.EnvManager(parallelProcessingFactor="70%"):  #transferencia do arquivo para um servido do INEA
    comp = rcc.map(compcic) #aplicando função de cópia